In this notebook we first do the classification using the BERT transformer
This is our first classification task.

The output classification vector from the transformer is saved to be used by the FCNN
This is our second classification task.

In [1]:
# Importing necessary libraries
import pandas as pd
from datetime import datetime
import sklearn
import torch
import torch.nn as nn
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [2]:
from simpletransformers.classification import ClassificationModel

## Preparing the dataset

Some pre-processing to the dataset has already been done in preparation for various tests, so this processing is not from scratch.

In [4]:
# procedure for getting the data sets and formatting them for the transformer
 

def prepareDataset( filename):
     
    ReadSet=pd.read_excel(filename )

    ReadSet['text']=ReadSet['Statement']
    ReadSet['labels']=ReadSet['Label']
    
    ReadSet=ReadSet.drop(['ID','Label','Statement','Subject','Speaker','Job','From','Affiliation','PantsTotal','NotRealTotal','BarelyTotal','HalfTotal','MostlyTotal','Truths','Context'
],axis=1)
     
    return ReadSet


In [5]:
# preparing the training dataset
train=prepareDataset( 'train.xlsx')
# and display for inspecting
train

,text,labels
0,The attorney general requires that rape victim...,0
1,President Clinton reduced the scale of our mil...,3
2,"I used tax cuts to help create over 80,000 job...",4
3,"New Mexico moved ""up to"" sixth in the nation i...",4
4,"Corporate profits are up, CEO pay is up, but a...",5
...,...,...
10264,"Under Obamacare, premiums have doubled and tri...",4
10265,We adopted the modern Social Security system a...,5
10266,More than two months ago President Barack Obam...,3
10267,"We had a massive landslide victory, as you kno...",1


In [6]:
# preparing the evaluation/validation dataset
Eval=prepareDataset('valid.xlsx')
# and display for inspecting
Eval

,text,labels
0,The president is brain-dead.,0
1,"Barack Obama supported keeping troops in Iraq,...",3
2,"He's leading by example, refusing contribution...",3
3,I'm the first person who really took up the is...,4
4,I built that border fence in San Diego...and i...,4
...,...,...
1279,CNN accidentally aired 30 minutes of pornograp...,1
1280,President Obamas American Recovery and Reinves...,2
1281,We (in Illinois) have the fifth-highest tax bu...,4
1282,Says Donald Trump won more counties than any c...,4


In [7]:
# preparing the test set dataset
test=prepareDataset('test.xlsx')
test

,text,labels
0,New Mexico was 46th in teacher pay (when he wa...,4
1,Barack Obama and Hillary Clinton have changed ...,3
2,I'll tell you what I can tell this country: If...,1
3,Tommy Thompson created the first school choice...,5
4,Fifty-six percent decline in overall crime. A ...,5
...,...,...
1278,"We have trade agreements with 20 countries, an...",1
1279,On Donald Trumps plan to cut federal funding t...,4
1280,"Black Lives Matter, who are attacking law enfo...",2
1281,Latina who enthusiastically supported Donald T...,0


## Setting up the transformer for fine tuning

This is where changes are done to optimise the model

The simpletransformers library is the quickest way to do this at the time of writing. 
For more information on the settings and their default value go here:
https://github.com/ThilinaRajapakse/simpletransformers#default-settings 

###### Please do read that reference before changing any parameters. Don't try to be a hero!

In [8]:
#Set the model being used here
model_class='bert'  # bert or roberta or albert
model_version='bert-base-cased' #bert-base-cased, roberta-base, roberta-large, albert-base-v2 OR albert-large-v2


output_folder='./TunedModels/'+model_class+'/'+model_version+"/"
cache_directory= "./TunedModels/"+model_class+"/"+model_version+"/"+"/cache/"
labels_count=6  # the number of classification classes

print('model variables were set up: ')

model variables were set up: 


In [9]:
# use this to test if writing to the directories is working

import os
print(os.getcwd())
print(output_folder)
print(cache_directory)

testWrite=train.head(30)
 
testWrite.to_csv(output_folder+'DeleteThisToo.tsv', sep='\t')
testWrite.to_csv(cache_directory+'DeleteThisToo.tsv', sep='\t')

del(testWrite)

G:\0 finalThesis\LIAR_Text
./TunedModels/bert/bert-base-cased/
./TunedModels/bert/bert-base-cased//cache/


In [12]:
 
save_every_steps=1285
# assuming training batch size of 8
# any number above 1284 saves the model only at every epoch
# Saving the model mid training very often will consume disk space fast

train_args={
    "output_dir":output_folder,
    "cache_dir":cache_directory,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'num_train_epochs': 2,
    "save_steps": save_every_steps, 
    "learning_rate": 1.8e-5,
    "train_batch_size": 64,
    "eval_batch_size": 16,
    "evaluate_during_training_steps": 312,
    "max_seq_length": 100,
    "n_gpu": 1,
}

# Create a ClassificationModel
model = ClassificationModel(model_class, model_version, num_labels=labels_count, args=train_args) 

# You can set class weights by using the optional weight argument

### Loading a saved model (based on above args{})

If you stopped training you can continue training from a previously saved check point.
The next cell allows you to load a model from any checkpoint.
The number of epochs in the train_args{} will be done and continue tuning from your checkpoint.

###### HOWEVER
It will overwrite previous checkpoints!
Example:  If you load an epoch-3 checkpoint, the epoch-1 checkpoint will be overwritten by the 4th epoch and it will be equivalent to a 4th epoch even if you have epoch-1 in the name.
###### SO BE CAREFUL

In [11]:
# loading a previously saved model based on this particular Transformer Class and model_name

# loading the checkpoint that gave the best result
CheckPoint='checkpoint-322-epoch-2'  #epoch 2


preSavedCheckpoint=output_folder+CheckPoint

print('Loading model, please wait...')
model = ClassificationModel( model_class, preSavedCheckpoint, num_labels=labels_count, args=train_args) 
print('model in use is :', preSavedCheckpoint )

Loading model, please wait...
model in use is : ./TunedModels/bert/bert-base-cased/checkpoint-322-epoch-2


## Training the Transformer

Skip the next cell if you want to skip the training and go directly to the evaluation

In [13]:
# Train the model
start_time = datetime.now()
model.train_model(train)
print("Training time: ", datetime.now() - start_time)

Converting to features started. Cache is not used.



Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 1.845521

C:\Users\Mark\Anaconda3\envs\thesis\lib\site-packages\torch\optim\lr_scheduler.py:110: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Seems like `optimizer.step()` has been overridden after learning rate scheduler "


Running loss: 1.764381

C:\Users\Mark\Anaconda3\envs\thesis\lib\site-packages\torch\optim\lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 1.747870


Running loss: 1.591926

Training of bert model complete. Saved to ./TunedModels/bert/bert-base-cased/.
Training time:  0:06:27.380125


## Evaluating the training

In [14]:
TrainResult, TrainModel_outputs, wrong_predictions = model.eval_model(train, acc=sklearn.metrics.accuracy_score)

EvalResult, EvalModel_outputs, wrong_predictions = model.eval_model(Eval, acc=sklearn.metrics.accuracy_score)


print('Training Result:', TrainResult['acc'])
#print('Model Out:', TrainModel_outputs)

print('Eval Result:', EvalResult['acc'])
#print('Model Out:', EvalModel_outputs)

print("Training & Evaluation time taken: ", datetime.now() - start_time)

Converting to features started. Cache is not used.



{'mcc': 0.16271697234721857, 'acc': 0.3192131658389327, 'eval_loss': 1.6220376242729733}
Converting to features started. Cache is not used.



{'mcc': 0.08116052192534284, 'acc': 0.2562305295950156, 'eval_loss': 1.687631521695926}
Training Result: 0.3192131658389327
Eval Result: 0.2562305295950156
Training & Evaluation time taken:  0:08:35.250750


In [15]:
TestResult, TestModel_outputs, wrong_predictions = model.eval_model(test, acc=sklearn.metrics.accuracy_score)

print('Test Set Result:', TestResult['acc'])
#print('Model Out:', TestModel_outputs)

Converting to features started. Cache is not used.



{'mcc': 0.10674957436746564, 'acc': 0.2766952455183164, 'eval_loss': 1.6706138746238048}
Test Set Result: 0.2766952455183164


In [16]:
 Pred=[]

countCorrect=0

for row in range(TestModel_outputs.shape[0]):
    outputs=TestModel_outputs[row]
    #print(test.iloc[row,0])
    print(outputs, end=' ')
    
    result=0
    if outputs[0]<outputs[1]:result=1
    if outputs[result]<outputs[2]:result=2
    if outputs[result]<outputs[3]:result=3
    if outputs[result]<outputs[4]:result=4
    if outputs[result]<outputs[5]:result=5
    Pred.append(result)
    print(result, ' ',test.iloc[row,1], end=' ')
    if result==test.iloc[row,1]:
        countCorrect+=1
        print('Match',countCorrect)
    print('')

print(countCorrect)

[-1.5859375  -0.17675781 -0.6533203   0.28076172  1.0107422   1.0078125 ] 4   4 Match 1

[-0.42333984  0.30200195  0.27197266  0.11785889 -0.3774414  -0.5854492 ] 1   3 
[-0.67822266  0.28149414 -0.13684082 -0.04931641 -0.28320312  0.02845764] 1   1 Match 2

[-1.1582031  -0.05422974  0.22033691  0.6225586   0.04275513 -0.2541504 ] 3   5 
[-1.5605469  -0.1184082  -0.30859375  0.6230469   0.99560547  0.46679688] 4   5 
[-1.4960938   0.23535156 -0.20532227  0.42089844  0.40698242  0.29785156] 3   2 
[-1.8173828  -0.08123779 -0.27124023  0.71533203  1.0253906   0.3005371 ] 4   4 Match 3

[-1.2617188  -0.0289917   0.15429688  0.31982422  0.27148438 -0.0657959 ] 3   5 
[-1.4570312   0.06799316 -0.31713867  0.44067383  0.7050781   0.71875   ] 5   4 
[-1.6611328   0.05957031 -0.19628906  0.6015625   0.8798828   0.4099121 ] 4   5 
[-1.1210938   0.12176514  0.22509766  0.6645508  -0.01420593 -0.45922852] 3   5 
[-1.1708984   0.27539062 -0.09887695  0.22485352  0.07519531  0.03710938] 1   3 
[-0.

[-1.3828125  -0.05761719  0.11505127  0.6118164   0.06762695 -0.03848267] 3   4 
[-1.7207031  -0.24768066 -0.22644043  0.76220703  0.7314453   0.46875   ] 3   5 
[-1.6347656  -0.09094238 -0.09320068  0.54003906  0.94628906  0.24414062] 4   1 
[-1.7802734  -0.14526367  0.05651855  0.9868164   0.5498047  -0.03082275] 3   2 
[-0.4013672   0.37768555 -0.40942383 -0.27490234 -0.2397461   0.09002686] 1   2 
[ 0.2277832   0.33642578  0.01015472 -0.48364258 -0.8598633  -0.44189453] 1   1 Match 33

[-0.83984375  0.22277832 -0.203125    0.24853516  0.3395996   0.14465332] 4   2 
[-1.2675781  -0.01068878  0.16381836  0.6689453   0.23413086 -0.20776367] 3   1 
[-0.72558594  0.27368164 -0.27783203 -0.11212158 -0.08703613  0.20654297] 1   2 
[-1.4482422  -0.17736816 -0.3173828   0.5493164   0.6748047   0.37719727] 4   5 
[-0.6381836   0.5136719  -0.28930664 -0.24951172 -0.19311523  0.14611816] 1   5 
[-1.3789062   0.08428955 -0.14428711  0.39892578  0.38232422  0.1270752 ] 3   2 
[-1.0478516  -0.061

[-0.18969727  0.22021484  0.23742676 -0.10852051 -0.4248047  -0.4555664 ] 2   2 Match 64

[ 0.28881836  0.44995117 -0.27978516 -0.64404297 -0.57128906 -0.09368896] 1   5 
[-0.1427002   0.17456055  0.40795898  0.22668457 -0.19189453 -0.6201172 ] 2   3 
[-0.9682617  -0.09442139  0.18237305  0.46777344  0.15466309 -0.15332031] 3   4 
[-1.3330078   0.02597046 -0.28686523  0.38330078  0.79345703  0.63720703] 4   1 
[-0.6152344   0.1875      0.20300293 -0.02572632  0.11468506 -0.17932129] 2   1 
[-1.4541016   0.03030396 -0.47753906  0.40112305  0.61328125  0.47216797] 4   1 
[-0.2890625   0.01476288 -0.1361084  -0.25830078 -0.35546875 -0.2631836 ] 1   0 
[-0.35424805  0.15332031  0.43603516  0.26342773 -0.45092773 -0.6152344 ] 2   4 
[-0.7285156   0.1685791   0.4321289   0.46923828 -0.04400635 -0.5234375 ] 3   1 
[-1.0595703   0.02639771  0.24621582  0.59033203 -0.12548828 -0.2944336 ] 3   3 Match 65

[-0.09130859  0.11883545  0.43920898  0.09545898 -0.4404297  -0.56689453] 2   1 
[-1.083007


[-1.7119141  -0.29638672 -0.13500977  1.0126953   0.6665039   0.24938965] 3   3 Match 104

[-0.46069336  0.07800293  0.35253906  0.27075195 -0.1381836  -0.47998047] 2   2 Match 105

[-0.90966797  0.42456055 -0.02626038  0.26782227 -0.01687622 -0.15637207] 1   1 Match 106

[-1.7958984   0.01930237  0.007164    0.55371094  0.49560547  0.3503418 ] 3   3 Match 107

[-0.61376953  0.18676758 -0.22131348 -0.03433228  0.02947998  0.01580811] 1   3 
[-0.37475586  0.01322174  0.49926758  0.28051758 -0.43139648 -0.6635742 ] 2   1 
[-0.66845703  1.2763672  -0.5078125   0.06878662 -0.4729004  -0.08947754] 1   1 Match 108

[ 0.11547852  0.40527344  0.30517578 -0.40185547 -0.57128906 -0.50341797] 1   1 Match 109

[ 0.03988647  0.45092773 -0.06500244 -0.30493164 -0.53808594 -0.09490967] 1   4 
[-1.1064453   0.08392334  0.28320312  0.6010742   0.17004395 -0.39331055] 3   3 Match 110

[-1.5380859   0.03787231  0.13098145  0.63427734  0.5566406  -0.04437256] 3   3 Match 111

[-0.46850586  0.79003906 -0.

[-0.90527344  0.27148438 -0.21191406 -0.07110596 -0.2861328   0.16174316] 1   1 Match 139

[-0.2788086   0.16699219  0.5004883   0.12890625 -0.64990234 -0.67871094] 2   2 Match 140

[-0.97998047  0.12817383  0.15344238  0.23901367 -0.02760315 -0.03744507] 3   4 
[-1.453125    0.07666016  0.17199707  0.48779297  0.48876953  0.0435791 ] 4   2 
[ 0.2644043   0.24389648 -0.04724121 -0.45361328 -0.6464844  -0.36206055] 0   2 
[-0.50683594  0.35083008 -0.23486328 -0.3203125  -0.49291992  0.03988647] 1   3 
[-1.3574219   0.21020508 -0.35888672  0.2932129   0.3798828   0.5151367 ] 5   5 Match 141

[-0.19396973  0.36816406  0.10253906 -0.08282471 -0.3330078  -0.3359375 ] 1   2 
[-1.2021484   0.09417725  0.08093262  0.5053711   0.5761719  -0.06976318] 4   0 
[-1.8242188  -0.15197754 -0.41992188  0.59277344  0.93359375  0.6738281 ] 4   4 Match 142

[-1.2460938   0.24560547  0.1463623   0.7524414   0.12988281 -0.46313477] 3   1 
[-0.04174805  0.4182129   0.36523438 -0.24829102 -0.6142578  -0.67871

...

[-1.7373047  -0.06958008  0.00954437  0.62646484  0.83984375  0.2770996 ] 4   5 
[-0.7451172   0.3149414   0.08880615  0.17163086 -0.30688477 -0.24768066] 1   2 
[-0.6040039   0.46240234 -0.20837402 -0.01296997 -0.20471191 -0.01307678] 1   2 
[-1.2011719   0.22021484 -0.00966644  0.41967773  0.11444092  0.00474548] 3   4 
[-1.4541016   0.01255035 -0.1899414   0.49438477  0.75878906  0.42138672] 4   1 
[-1.5791016   0.20129395 -0.26489258  0.2763672   0.6411133   0.6381836 ] 4   4 Match 215

[-0.3239746   0.22717285 -0.22680664 -0.12780762 -0.09222412 -0.22338867] 1   3 
[-1.4560547  -0.21606445 -0.16003418  0.4580078   0.79541016  0.3876953 ] 4   4 Match 216

[-0.54541016  1.3369141  -0.640625   -0.17626953 -0.7451172  -0.16699219] 1   5 
[-1.3486328   0.30200195 -0.03173828  0.37426758  0.3395996   0.00933838] 3   2 
[-1.2021484   0.12670898 -0.11669922  0.5136719   0.44799805  0.07434082] 3   5 
[-1.328125    0.05032349 -0.18383789  0.36499023  0.6069336   0.21447754] 4   3 
[-1.5048

[-0.38598633  0.21618652  0.41870117  0.19421387 -0.5292969  -0.67333984] 2   2 Match 254

[-0.37963867  0.14221191  0.35058594  0.2956543  -0.04421997 -0.5444336 ] 2   0 
[-0.9707031   0.23498535 -0.10961914  0.25170898  0.2524414   0.00871277] 4   4 Match 255

[-1.4824219  -0.15405273 -0.23181152  0.45336914  0.7832031   0.36035156] 4   3 
[-1.0537109   0.30151367 -0.35009766  0.14855957  0.6088867   0.35302734] 4   5 
[-0.5083008   0.29492188  0.27563477  0.22351074 -0.50878906 -0.61572266] 1   2 
[-0.07781982  0.75683594 -0.40844727 -0.3635254  -0.29125977 -0.1821289 ] 1   1 Match 256

[-0.5444336   0.3256836  -0.02482605 -0.02157593 -0.41064453 -0.17810059] 1   2 
[-1.7568359  -0.05871582 -0.3190918   0.4716797   1.0009766   0.7421875 ] 4   4 Match 257

[-0.26757812  0.48510742 -0.40454102 -0.4404297  -0.11291504  0.19946289] 1   5 
[ 0.12939453  0.46264648  0.22509766 -0.18762207 -0.55078125 -0.5571289 ] 1   5 
[-1.6220703  -0.09606934 -0.06817627  0.6845703   0.88183594  0.17773

[-0.80371094  0.10839844  0.39135742  0.46142578 -0.05352783 -0.5678711 ] 3   3 Match 273

[-0.44726562  0.02236938  0.30932617  0.10687256 -0.31713867 -0.4165039 ] 2   4 
[-1.0585938   0.13793945  0.15344238  0.51660156  0.14453125 -0.22094727] 3   3 Match 274

[ 0.12915039  0.30004883  0.13586426 -0.14294434 -0.48535156 -0.43823242] 1   1 Match 275

[-0.27416992  0.5449219  -0.12792969 -0.31835938 -0.42236328 -0.07568359] 1   1 Match 276

[-1.1689453   0.22924805 -0.10369873  0.22851562 -0.05895996  0.16833496] 1   1 Match 277

[-1.2509766   0.02453613 -0.46875     0.05273438  0.671875    0.6723633 ] 5   5 Match 278

[ 0.00935364  0.32006836  0.09533691 -0.3569336  -0.7451172  -0.3359375 ] 1   0 
[-1.8466797  -0.22888184 -0.15368652  0.88964844  0.79345703  0.33740234] 3   1 
[-0.81933594  0.10614014  0.32592773  0.32226562 -0.13720703 -0.31469727] 2   3 
[-0.7475586   0.28442383 -0.29858398 -0.22631836  0.17614746  0.50341797] 5   1 
[-0.5571289   1.1699219  -0.39648438 -0.1459961  

[-1.4355469  -0.1385498  -0.36791992  0.32348633  0.9140625   0.5942383 ] 4   4 Match 311

[-0.85009766  0.25585938  0.2064209   0.18786621 -0.07818604 -0.1928711 ] 1   1 Match 312

[-1.7167969  -0.15197754  0.09661865  0.9736328   0.38232422 -0.18395996] 3   2 
[-1.5957031  -0.14807129 -0.28833008  0.6767578   0.69921875  0.44384766] 4   1 
[-0.21069336  0.22460938  0.10083008 -0.21984863 -0.6176758  -0.25708008] 1   4 
[-0.43701172 -0.06298828  0.24890137  0.06536865 -0.32348633 -0.3737793 ] 2   1 
[-0.31396484  0.19519043  0.21435547  0.05688477 -0.328125   -0.50390625] 2   2 Match 313

[-1.3164062   0.05044556 -0.06524658  0.4020996   0.5151367   0.19274902] 4   5 
[-1.7509766  -0.19628906 -0.1706543   0.8417969   0.98291016  0.19812012] 4   2 
[-1.5097656   0.08880615  0.15124512  0.66015625  0.03347778 -0.0526123 ] 3   3 Match 314

[-1.5439453   0.07110596 -0.3762207   0.47485352  0.9399414   0.52197266] 4   4 Match 315

[-0.82714844  0.13317871  0.48217773  0.44995117  0.1466064

[-0.53466797  0.2788086   0.03695679  0.04266357 -0.50927734 -0.30200195] 1   2 
[-1.6894531  -0.06945801 -0.0075531   0.96728516  0.5683594  -0.01358795] 3   4 
[-1.46875    -0.10394287 -0.5263672   0.18249512  0.80859375  0.84716797] 5   4 
[-0.7480469   0.3022461  -0.11376953  0.11016846 -0.3852539  -0.03207397] 1   1 Match 336

[-3.77297401e-05  2.64160156e-01 -1.23291016e-01 -3.58886719e-01
 -3.75244141e-01 -1.75048828e-01] 1   0 
[-1.4873047  -0.17077637 -0.31225586  0.49804688  0.8300781   0.47338867] 4   4 Match 337

[-0.55029297  0.20336914  0.45336914  0.31396484 -0.28100586 -0.5073242 ] 2   3 
[-1.1005859   0.3083496  -0.12695312  0.29589844 -0.0824585  -0.01942444] 1   1 Match 338

[-0.8027344   0.18249512  0.35107422  0.42163086 -0.2277832  -0.4074707 ] 3   2 
[-1.8320312  -0.13891602 -0.3071289   0.6064453   1.0429688   0.59814453] 4   3 
[-1.6533203  -0.29174805 -0.30737305  0.7265625   0.9770508   0.515625  ] 4   3 
[-0.9448242   0.08581543 -0.16564941  0.03598022  0.24

In [17]:
from sklearn import metrics
 
print(metrics.confusion_matrix(test['labels'],Pred))

[[  4  44  17  13  11   3]
 [  2 112  23  54  46  13]
 [  4  63  36  62  41   8]
 [  1  74  22  99  62   9]
 [  1  43  16  79  84  26]
 [  0  52   5  53  81  20]]


In [18]:
target_names = ['Pants', 'False', 'Barely-True','Half-True','Mostly-True','True']

print(metrics.classification_report(test['labels'], Pred,target_names =target_names))

              precision    recall  f1-score   support

       Pants       0.33      0.04      0.08        92
       False       0.29      0.45      0.35       250
 Barely-True       0.30      0.17      0.22       214
   Half-True       0.28      0.37      0.32       267
 Mostly-True       0.26      0.34      0.29       249
        True       0.25      0.09      0.14       211

    accuracy                           0.28      1283
   macro avg       0.29      0.24      0.23      1283
weighted avg       0.28      0.28      0.26      1283



In [19]:
# saving the output of the models to CSVs
#these are 1X6 classification vectors

SavesDirectory='./TunedModels/'+model_class+'/'+model_version+"/Saves/"
print('Saving...')
trainOut = pd.DataFrame(data= TrainModel_outputs )
trainOut.to_csv(SavesDirectory+'trainOut.tsv', sep='\t',  index=False)

evalOut = pd.DataFrame(data= EvalModel_outputs )
evalOut.to_csv(SavesDirectory+'evalOut.tsv', sep='\t',  index=False)

testOut = pd.DataFrame(data= TestModel_outputs )
testOut.to_csv(SavesDirectory+'testOut.tsv', sep='\t',  index=False)

print('Saving Complete on',datetime.now() ,'in:', SavesDirectory)

Saving...
Saving Complete on 2020-05-01 09:43:39.161107 in: ./TunedModels/bert/bert-base-cased/Saves/


In [20]:
del(model)
del(train,Eval,test)
del(trainOut,evalOut,testOut)
torch.cuda.empty_cache()

#  Adding the reputation vector

This section takes the output results from the transformer used above and uses it together with the speaker's reputation to enhance the classification.

Before running this section it is suggested that you halt the program and start running it again from this cell.
The neural net will likely have an error caused by some unreleased variable used by thr simple transformers library.

In [1]:
import pandas as pd
import torch
import torch.nn as nn
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:

train=pd.read_excel('trainReputation.xlsx' )
train=train.iloc[:,:-2].astype(float)
train=train/200  #for scaling
#train

model_class='bert'  # bert or roberta or albert
model_version='bert-base-cased' #bert-base-cased, roberta-base, roberta-large, albert-base-v2 OR albert-large-v2
SavesDirectory='./TunedModels/'+model_class+'/'+model_version+"/Saves/"
TF_Output=pd.read_csv( SavesDirectory+'trainOut.tsv', sep='\t')

train=pd.concat([train,TF_Output], axis=1)

train

,PantsTotal,NotRealTotal,BarelyTotal,HalfTotal,MostlyTotal,Truths,0,1,2,3,4,5
0,0.005,0.000,0.000,0.000,0.000,0.000,-0.533691,0.247803,-0.193848,-0.126465,-0.089417,0.063416
1,0.095,0.160,0.170,0.290,0.165,0.165,-1.366211,-0.036041,0.095459,0.850098,0.082397,-0.231812
2,0.005,0.010,0.005,0.015,0.040,0.010,-1.458984,-0.033417,0.235718,0.880859,0.232178,-0.344482
3,0.005,0.010,0.005,0.015,0.040,0.010,-1.730469,-0.323242,-0.382812,0.850098,0.841309,0.455322
4,0.035,0.145,0.200,0.345,0.380,0.365,-1.690430,-0.112061,-0.055603,0.768555,0.890137,0.208252
...,...,...,...,...,...,...,...,...,...,...,...,...
10264,0.005,0.030,0.070,0.050,0.050,0.020,-1.382812,-0.120605,0.241333,0.741211,0.223022,0.005543
10265,0.055,0.075,0.080,0.100,0.050,0.035,-1.219727,0.326416,0.036774,0.218262,0.193237,0.149048
10266,0.035,0.115,0.140,0.190,0.170,0.075,-1.498047,-0.096313,0.275879,0.799316,0.517090,-0.206299
10267,0.305,0.570,0.315,0.255,0.185,0.070,-1.075195,0.350342,-0.005589,0.159546,-0.111633,0.172729


In [3]:
TrainLables=pd.read_excel('trainReputation.xlsx' )
TrainLables=TrainLables.iloc[:,-1] 

TrainLables=pd.get_dummies(TrainLables)
TrainLables

,0,1,2,3,4,5
0,1,0,0,0,0,0
1,0,0,0,1,0,0
2,0,0,0,0,1,0
3,0,0,0,0,1,0
4,0,0,0,0,0,1
...,...,...,...,...,...,...
10264,0,0,0,0,1,0
10265,0,0,0,0,0,1
10266,0,0,0,1,0,0
10267,0,1,0,0,0,0


In [4]:
input=torch.tensor(train.values)
input

tensor([[ 0.0050,  0.0000,  0.0000,  ..., -0.1265, -0.0894,  0.0634],
        [ 0.0950,  0.1600,  0.1700,  ...,  0.8501,  0.0824, -0.2318],
        [ 0.0050,  0.0100,  0.0050,  ...,  0.8809,  0.2322, -0.3445],
        ...,
        [ 0.0350,  0.1150,  0.1400,  ...,  0.7993,  0.5171, -0.2063],
        [ 0.3050,  0.5700,  0.3150,  ...,  0.1595, -0.1116,  0.1727],
        [ 0.0000,  0.0050,  0.0000,  ..., -0.2220, -0.4087, -0.5107]],
       dtype=torch.float64)

In [5]:
targets=torch.tensor(TrainLables.astype(float).values)

targets

tensor([[1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        ...,
        [0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.]], dtype=torch.float64)

In [6]:
 
size= torch.tensor(input[0].size())
InputSize=size.item()

OutputSize=torch.tensor(targets[0].size()).item()

print('input size:', InputSize)
print('output size:', OutputSize)

input size: 12
output size: 6


In [7]:

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        
         
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(InputSize, 24)  # input size 32
        self.fc2 = nn.Linear(24, 12)
        self.fc3 = nn.Linear(12, OutputSize)  #classifies 'outputsize' different classes

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x)) 
        x = torch.tanh(self.fc3(x)).double()
        return x

    

#now we use it

net = Net()

In [8]:
# here we  setup the neural network parameters
# pick an optimizer (Simple Gradient Descent)

learning_rate = 9e-4
criterion = nn.MSELoss()  #computes the loss Function

import torch.optim as optim

# creating optimizer
#optimizer = optim.SGD(net.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)


In [20]:
for epoch in range(1000):  
        
    optimizer.zero_grad()   # zero the gradient buffers
    output = net(input.float())

    loss = criterion(output, targets)
    print('Loss:', loss, ' at epoch:', epoch)

    loss.backward()  #backprop
    optimizer.step()    # Does the update

Loss: tensor(0.1261, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 0
Loss: tensor(0.1261, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 1
Loss: tensor(0.1261, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2
Loss: tensor(0.1261, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 3
Loss: tensor(0.1261, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 4
Loss: tensor(0.1261, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 5
Loss: tensor(0.1261, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 6
Loss: tensor(0.1261, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7
Loss: tensor(0.1261, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 8
Loss: tensor(0.1261, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 9
Loss: tensor(0.1261, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 10
Loss: tensor(0.1261, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 11
Loss: tensor(0

Loss: tensor(0.1259, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 102
Loss: tensor(0.1259, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 103
Loss: tensor(0.1259, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 104
Loss: tensor(0.1259, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 105
Loss: tensor(0.1259, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 106
Loss: tensor(0.1259, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 107
Loss: tensor(0.1259, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 108
Loss: tensor(0.1259, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 109
Loss: tensor(0.1259, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 110
Loss: tensor(0.1259, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 111
Loss: tensor(0.1259, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 112
Loss: tensor(0.1259, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epo

Loss: tensor(0.1257, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 209
Loss: tensor(0.1257, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 210
Loss: tensor(0.1257, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 211
Loss: tensor(0.1257, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 212
Loss: tensor(0.1257, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 213
Loss: tensor(0.1257, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 214
Loss: tensor(0.1257, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 215
Loss: tensor(0.1257, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 216
Loss: tensor(0.1257, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 217
Loss: tensor(0.1257, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 218
Loss: tensor(0.1257, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 219
Loss: tensor(0.1257, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epo

Loss: tensor(0.1254, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 320
Loss: tensor(0.1254, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 321
Loss: tensor(0.1254, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 322
Loss: tensor(0.1254, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 323
Loss: tensor(0.1254, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 324
Loss: tensor(0.1254, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 325
Loss: tensor(0.1254, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 326
Loss: tensor(0.1254, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 327
Loss: tensor(0.1254, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 328
Loss: tensor(0.1254, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 329
Loss: tensor(0.1254, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 330
Loss: tensor(0.1254, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epo

Loss: tensor(0.1252, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 429
Loss: tensor(0.1252, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 430
Loss: tensor(0.1252, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 431
Loss: tensor(0.1252, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 432
Loss: tensor(0.1252, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 433
Loss: tensor(0.1252, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 434
Loss: tensor(0.1252, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 435
Loss: tensor(0.1252, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 436
Loss: tensor(0.1252, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 437
Loss: tensor(0.1252, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 438
Loss: tensor(0.1252, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 439
Loss: tensor(0.1252, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epo

...

Loss: tensor(0.0960, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7433
Loss: tensor(0.0960, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7434
Loss: tensor(0.0960, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7435
Loss: tensor(0.0960, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7436
Loss: tensor(0.0960, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7437
Loss: tensor(0.0960, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7438
Loss: tensor(0.0960, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7439
Loss: tensor(0.0960, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7440
Loss: tensor(0.0960, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7441
Loss: tensor(0.0960, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7442
Loss: tensor(0.0960, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7443
Loss: tensor(0.0960, dtype=torch.float64, grad_fn=<MseLossBackwar

Loss: tensor(0.0959, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7553
Loss: tensor(0.0959, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7554
Loss: tensor(0.0959, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7555
Loss: tensor(0.0959, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7556
Loss: tensor(0.0959, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7557
Loss: tensor(0.0959, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7558
Loss: tensor(0.0959, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7559
Loss: tensor(0.0959, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7560
Loss: tensor(0.0959, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7561
Loss: tensor(0.0959, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7562
Loss: tensor(0.0959, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7563
Loss: tensor(0.0959, dtype=torch.float64, grad_fn=<MseLossBackwar

Loss: tensor(0.0959, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7674
Loss: tensor(0.0959, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7675
Loss: tensor(0.0959, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7676
Loss: tensor(0.0959, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7677
Loss: tensor(0.0959, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7678
Loss: tensor(0.0959, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7679
Loss: tensor(0.0959, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7680
Loss: tensor(0.0959, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7681
Loss: tensor(0.0959, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7682
Loss: tensor(0.0959, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7683
Loss: tensor(0.0959, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7684
Loss: tensor(0.0959, dtype=torch.float64, grad_fn=<MseLossBackwar

Loss: tensor(0.0958, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7796
Loss: tensor(0.0958, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7797
Loss: tensor(0.0958, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7798
Loss: tensor(0.0958, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7799
Loss: tensor(0.0958, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7800
Loss: tensor(0.0958, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7801
Loss: tensor(0.0958, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7802
Loss: tensor(0.0958, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7803
Loss: tensor(0.0958, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7804
Loss: tensor(0.0958, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7805
Loss: tensor(0.0958, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7806
Loss: tensor(0.0958, dtype=torch.float64, grad_fn=<MseLossBackwar

Loss: tensor(0.0958, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7918
Loss: tensor(0.0958, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7919
Loss: tensor(0.0958, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7920
Loss: tensor(0.0958, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7921
Loss: tensor(0.0958, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7922
Loss: tensor(0.0958, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7923
Loss: tensor(0.0958, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7924
Loss: tensor(0.0958, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7925
Loss: tensor(0.0958, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7926
Loss: tensor(0.0958, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7927
Loss: tensor(0.0958, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7928
Loss: tensor(0.0958, dtype=torch.float64, grad_fn=<MseLossBackwar

In [10]:
#save the FCNN model

stage='NNetwork/'
SavesDirectory='./TunedModels/'+model_class+'/'+model_version+"/"+stage
#PATH = SavesDirectory+'Tanh_MSE_adam4801.pth'

torch.save(net.state_dict(), PATH)

# more on saving pytorch networks: https://pytorch.org/docs/stable/notes/serialization.html

NameError: name 'PATH' is not defined

In [ ]:
#load previously saved FCNN model 

stage='NNetwork/'
SavesDirectory='./TunedModels/'+model_class+'/'+model_version+"/"+stage
PATH = SavesDirectory+'Tanh_MSE_adam4801.pth'

net = Net()
net.load_state_dict(torch.load(PATH))

In [21]:
correct = 0
total = 0

countCorrect0=0
countCorrect1=0
count0=0
count1=0
labels=pd.read_excel('trainReputation.xlsx' )

Y=[]  #target
Pred=[]  #predicted

with torch.no_grad():
    for row in range(len(input)):
        outputs = net(input[row,:].float())
        result=0
        total+=1
        if outputs[0]<outputs[1]:result=1
        if outputs[result]<outputs[2]:result=2
        if outputs[result]<outputs[3]:result=3
        if outputs[result]<outputs[4]:result=4
        if outputs[result]<outputs[5]:result=5
        
        if TrainLables.iloc[row,result]==1: correct+=1
        
        Y.append(labels.iloc[row])
        Pred.append(result)
        
        print(result, end=' ')
        
    
print('Correct:', correct, 'out of:', total )
print('Accuracy of the network : ',( 100 * correct / total))


0 3 4 4 3 1 4 2 2 1 3 4 4 5 3 5 1 5 3 3 5 1 5 4 3 2 4 1 3 1 4 1 1 3 5 3 4 4 5 4 3 5 4 4 5 5 3 2 5 1 5 4 3 1 4 3 3 3 1 2 1 4 5 1 2 3 4 5 4 3 4 3 2 1 5 4 4 3 3 5 3 3 4 3 3 3 3 5 1 1 3 3 3 0 0 1 4 2 3 1 1 4 4 4 4 2 4 3 5 3 4 4 1 1 4 3 2 3 4 3 4 3 5 5 0 3 3 3 1 1 3 3 1 5 4 1 1 1 4 1 1 2 1 3 3 3 4 3 5 2 2 3 1 5 2 1 3 3 4 1 5 4 3 1 1 0 1 2 3 3 3 5 4 4 1 5 5 5 0 3 4 2 3 4 3 2 3 2 4 3 1 2 3 4 4 5 2 4 3 3 2 2 0 5 1 1 1 4 2 2 3 0 0 0 1 3 2 4 4 5 3 3 3 3 1 2 0 3 0 3 1 4 1 1 2 3 5 5 5 2 1 5 3 3 3 1 3 4 4 3 3 3 4 3 2 3 3 5 5 4 2 0 5 4 1 1 5 3 4 1 5 3 4 1 5 0 0 0 3 5 3 0 2 1 1 3 4 3 4 2 4 2 4 4 2 1 3 1 1 1 4 1 1 3 1 4 3 4 4 1 0 0 0 1 4 4 5 5 3 0 3 1 4 1 5 1 1 5 2 1 3 1 1 3 2 1 1 5 3 2 4 4 4 4 4 1 4 3 2 3 3 3 2 5 3 0 1 3 5 2 0 4 4 4 4 4 4 4 4 3 4 1 1 5 4 1 1 1 5 4 2 3 1 3 1 3 4 3 2 1 5 1 0 0 1 4 1 4 4 4 1 4 5 0 3 3 3 2 4 4 3 3 3 3 3 2 3 5 3 3 4 1 4 3 3 0 4 1 1 5 3 3 1 0 3 4 1 0 3 2 2 1 2 4 4 5 5 4 1 1 1 3 1 3 3 1 1 1 3 1 0 2 0 1 1 2 5 3 1 1 1 1 0 1 1 4 4 3 2 3 2 1 3 2 4 3 3 0 4 1 4 1 1 1 4 1 1 3 3 3 

 3 1 5 2 3 3 1 3 2 5 4 3 2 4 3 4 3 4 5 3 3 5 2 1 0 4 2 2 1 5 3 0 4 1 4 4 0 4 3 1 4 3 3 1 3 2 1 3 5 3 4 2 4 1 3 3 4 5 5 4 1 4 3 4 4 4 3 2 3 1 3 5 4 5 4 1 3 1 2 3 1 3 2 5 2 2 3 0 0 2 5 1 1 1 5 1 2 3 2 0 1 5 4 4 1 2 4 2 4 2 5 4 4 2 3 5 5 3 1 2 0 3 3 1 5 1 3 3 4 1 2 3 1 2 5 4 3 3 1 3 3 4 3 3 5 2 3 0 3 5 2 3 4 4 3 1 4 5 2 3 2 1 3 3 3 1 2 3 2 3 4 5 4 4 1 3 1 2 3 3 5 3 0 4 1 4 5 3 5 5 2 5 5 3 4 2 3 3 3 5 4 3 4 3 4 2 3 2 3 1 1 4 2 1 3 1 1 1 4 5 5 3 2 0 1 4 3 5 3 4 2 0 4 5 3 1 5 2 3 5 3 2 2 3 0 3 4 4 2 4 1 4 1 3 3 1 5 4 1 4 5 4 5 4 3 3 4 5 5 5 4 3 5 5 2 4 1 2 4 1 1 3 3 2 3 1 2 1 3 3 2 4 4 3 0 3 2 4 1 1 1 2 3 1 1 0 1 2 1 3 1 5 3 4 1 5 1 0 4 1 2 4 3 2 4 3 1 1 3 2 1 2 2 4 2 3 1 1 2 2 3 1 1 4 2 2 3 3 4 3 2 1 1 2 2 2 5 4 3 5 0 4 1 3 4 5 1 4 3 3 3 4 3 3 3 4 5 3 3 3 3 2 3 3 4 3 2 1 1 4 5 3 3 3 0 4 5 4 3 3 1 3 1 2 1 1 2 1 3 1 3 5 5 5 4 3 5 5 2 3 3 3 3 4 1 4 2 3 2 5 5 2 5 1 4 2 1 2 1 2 1 4 1 4 1 1 1 5 1 5 3 4 4 2 1 3 2 3 5 1 3 0 3 5 4 5 4 4 2 1 3 1 3 3 0 1 5 5 4 5 3 3 2 1 0 0 3 4 5 2 3 1 1 5 3 1 4 3 4 1

 3 1 1 0 4 0 1 4 3 3 3 4 2 4 1 5 4 1 4 3 2 4 5 4 5 4 1 2 1 1 3 4 3 2 2 3 5 0 2 1 2 4 1 3 4 3 3 3 5 3 2 3 0 1 4 1 1 4 1 5 5 3 4 0 3 3 3 5 4 1 4 3 5 0 0 4 4 4 4 2 4 1 2 1 3 2 5 1 2 3 4 3 3 1 4 5 4 3 4 1 2 0 0 2 5 1 3 1 1 1 3 1 3 1 5 1 1 4 1 0 1 3 3 3 1 0 3 2 1 3 4 5 4 3 2 4 1 5 1 4 3 4 1 1 5 5 4 3 1 1 3 2 4 4 1 1 1 4 4 1 3 1 3 5 4 1 0 4 2 1 2 1 5 1 4 0 4 0 2 1 1 4 5 1 0 1 4 3 3 3 1 4 4 1 1 1 4 1 5 4 3 5 4 3 3 3 2 1 5 5 2 1 5 1 2 4 1 1 1 5 5 1 2 3 1 2 1 1 2 1 3 4 1 2 3 2 2 5 5 2 1 2 1 1 1 1 4 1 4 5 3 3 2 4 2 1 4 4 1 2 3 2 4 2 2 1 2 5 2 4 2 1 1 3 2 3 1 2 1 1 3 2 2 4 1 2 2 1 0 4 4 3 2 4 3 5 4 1 4 1 4 5 5 4 2 1 1 5 3 2 5 4 4 4 5 5 5 3 2 4 1 3 1 2 3 0 0 1 2 2 0 5 1 3 1 4 4 1 1 4 1 5 4 2 4 4 4 1 3 3 5 1 4 1 4 3 1 3 1 1 3 5 3 4 4 1 4 2 4 1 1 4 4 2 5 1 4 1 4 2 1 0 4 4 1 5 1 1 4 0 4 2 1 4 3 5 2 1 2 5 4 4 5 2 3 3 4 1 4 2 0 3 5 1 3 3 1 3 0 4 1 4 2 2 5 1 2 1 5 3 5 5 5 2 1 1 1 1 1 1 3 3 1 1 4 3 2 4 1 1 3 1 1 4 3 3 3 3 2 4 3 1 1 4 1 5 2 0 1 2 1 5 3 1 1 2 2 4 3 3 4 4 4 4 5 2 1 4 4 4 2 2 1 4 3 1 2 2 1 4

In [12]:
# load the validation data

ValidData=pd.read_excel('validReputation.xlsx' )
ValidData=ValidData.iloc[:,:-2].astype(float)
ValidData=ValidData/200

SavesDirectory='./TunedModels/'+model_class+'/'+model_version+"/Saves/"
TF_Output=pd.read_csv( SavesDirectory+'evalOut.tsv', sep='\t')

ValidData=pd.concat([ValidData,TF_Output], axis=1)


ValidData=torch.tensor(ValidData.values)
ValidData

tensor([[ 0.0200,  0.0500,  0.0550,  ..., -0.4009, -0.7607, -0.2063],
        [ 0.0100,  0.0300,  0.0300,  ...,  0.2151, -0.2423, -0.4741],
        [ 0.0450,  0.3550,  0.3500,  ...,  0.3970, -0.1581, -0.2169],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.3386,  1.2178,  0.9692],
        [ 0.0000,  0.0500,  0.0400,  ...,  0.5190,  0.7275,  0.4744],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.6270,  0.7715,  0.3784]],
       dtype=torch.float64)

In [22]:
labels=pd.read_excel('validReputation.xlsx' )

labels=labels.iloc[:,-1] 
labelsOneHot=pd.get_dummies(labels)
labelsOneHot

,0,1,2,3,4,5
0,1,0,0,0,0,0
1,0,0,0,1,0,0
2,0,0,0,1,0,0
3,0,0,0,0,1,0
4,0,0,0,0,1,0
...,...,...,...,...,...,...
1279,0,1,0,0,0,0
1280,0,0,1,0,0,0
1281,0,0,0,0,1,0
1282,0,0,0,0,1,0


In [23]:
ValidLables =torch.tensor(labelsOneHot.values)
ValidLables

tensor([[1, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0, 0],
        ...,
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 0]], dtype=torch.uint8)

In [24]:
correct = 0
total = 0

countCorrect0=0
countCorrect1=0
count0=0
count1=0

Y=[]  #target
Pred=[]  #predicted

with torch.no_grad():
    for row in range(len(ValidData)):
        outputs = net(ValidData[row,:].float())
        result=0
        total+=1
        if outputs[0]<outputs[1]:result=1
        if outputs[result]<outputs[2]:result=2
        if outputs[result]<outputs[3]:result=3
        if outputs[result]<outputs[4]:result=4
        if outputs[result]<outputs[5]:result=5
        
        if labelsOneHot.iloc[row,result]==1: correct+=1
        
        Y.append(labels.iloc[row])
        Pred.append(result)
        
        print(result, end=' ')
        
    
print('Correct:', correct, 'out of:', total )
print('Accuracy of the network : ',( 100 * correct / total))

1 3 2 5 4 1 5 3 4 1 1 3 1 2 1 3 4 4 0 3 5 1 5 1 3 2 5 4 3 4 4 1 4 1 1 0 2 4 1 0 0 4 1 0 4 1 2 1 4 3 4 5 0 1 3 1 4 4 4 5 3 1 1 3 3 3 1 4 1 2 4 2 1 0 5 4 3 1 3 1 1 2 1 3 4 4 3 3 1 1 1 2 5 3 3 1 2 1 0 3 0 2 1 4 5 1 5 0 1 1 1 1 4 1 1 1 1 2 1 1 5 1 3 0 3 3 1 4 5 3 4 3 5 3 5 2 3 2 3 3 3 1 4 3 3 3 1 3 3 4 1 5 2 1 5 2 5 2 3 2 3 5 2 4 5 4 2 1 1 3 1 3 1 2 1 4 5 4 5 5 1 5 5 2 2 2 4 1 4 4 0 4 3 2 4 3 2 4 4 1 3 1 3 3 3 2 4 4 3 2 2 1 1 1 1 3 3 4 2 1 4 3 1 3 4 2 2 1 2 2 3 3 2 2 1 2 3 2 1 2 3 5 2 3 3 4 3 5 5 3 2 4 1 1 2 2 1 3 4 5 5 5 3 2 5 2 2 3 3 1 2 1 4 4 4 1 5 1 5 5 4 1 1 4 3 1 1 0 3 1 1 1 1 4 4 1 3 1 2 3 4 5 3 5 0 3 5 3 3 4 4 3 4 2 4 3 5 1 5 3 5 2 0 2 1 1 4 4 1 3 4 2 3 5 4 2 4 3 1 4 1 1 1 1 4 0 2 3 3 2 1 4 3 3 2 1 3 1 2 2 2 4 4 2 2 5 3 3 3 1 4 3 3 3 2 1 5 5 2 1 4 5 2 1 5 3 5 3 5 4 1 1 5 0 4 3 1 2 1 3 4 4 3 5 5 5 1 1 4 3 2 1 4 3 2 4 4 3 4 1 1 2 4 4 1 3 2 1 1 2 3 0 5 3 1 0 5 5 1 1 3 4 4 5 4 5 2 3 3 3 5 2 3 3 3 5 1 2 2 1 2 4 3 3 1 1 3 1 5 3 4 1 4 5 4 3 5 1 3 4 1 0 1 3 2 3 3 3 1 2 1 4 5 1 2 2 3 1 1 2 

In [16]:
# load the test data

TestData=pd.read_excel('testReputation.xlsx' )
TestData=TestData.iloc[:,:-2].astype(float)
TestData=TestData/200

SavesDirectory='./TunedModels/'+model_class+'/'+model_version+"/Saves/"
TF_Output=pd.read_csv( SavesDirectory+'testOut.tsv', sep='\t')

TestData=pd.concat([TestData,TF_Output], axis=1)


TestData=torch.tensor(TestData.values)
TestData

tensor([[ 0.0050,  0.0100,  0.0050,  ...,  0.2808,  1.0107,  1.0078],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.1179, -0.3774, -0.5854],
        [ 0.0200,  0.0250,  0.0600,  ..., -0.0493, -0.2832,  0.0285],
        ...,
        [ 0.0100,  0.0050,  0.0250,  ..., -0.5962, -0.7461, -0.4854],
        [ 0.2200,  0.0950,  0.0350,  ..., -0.4175, -0.6655, -0.1862],
        [ 0.0050,  0.0600,  0.0100,  ..., -0.0349, -0.1351, -0.0563]],
       dtype=torch.float64)

In [25]:
labels=pd.read_excel('testReputation.xlsx' )

labels=labels.iloc[:,-1] 
labelsOneHot=pd.get_dummies(labels)
labelsOneHot

,0,1,2,3,4,5
0,0,0,0,0,1,0
1,0,0,0,1,0,0
2,0,1,0,0,0,0
3,0,0,0,0,0,1
4,0,0,0,0,0,1
...,...,...,...,...,...,...
1278,0,1,0,0,0,0
1279,0,0,0,0,1,0
1280,0,0,1,0,0,0
1281,1,0,0,0,0,0


In [26]:
TestLables =torch.tensor(labelsOneHot.values)
TestLables

tensor([[0, 0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0, 0],
        ...,
        [0, 0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0]], dtype=torch.uint8)

In [27]:
correct = 0
total = 0

countCorrect0=0
countCorrect1=0
count0=0
count1=0

Y=[]  #target
Pred=[]  #predicted

with torch.no_grad():
    for row in range(len(TestData)):
        outputs = net(TestData[row,:].float())
        result=0
        total+=1
        if outputs[0]<outputs[1]:result=1
        if outputs[result]<outputs[2]:result=2
        if outputs[result]<outputs[3]:result=3
        if outputs[result]<outputs[4]:result=4
        if outputs[result]<outputs[5]:result=5
        
        if labelsOneHot.iloc[row,result]==1: correct+=1
        
        Y.append(labels.iloc[row])
        Pred.append(result)
        
        print(result, end=' ')
        
    
print('Correct:', correct, 'out of:', total )
print('Accuracy of the network : ',( 100 * correct / total))

4 3 2 2 3 1 4 4 5 4 3 1 1 1 3 3 1 1 1 4 1 4 3 3 1 5 4 4 3 5 1 1 0 3 5 4 5 4 4 4 1 5 5 1 1 3 4 4 1 1 4 4 3 3 1 2 5 5 5 1 1 4 1 3 1 4 3 5 3 4 3 1 2 5 1 1 4 1 1 2 3 1 1 4 1 4 1 4 1 4 1 1 1 1 2 1 3 1 3 2 4 0 3 1 1 1 1 3 1 3 1 3 3 3 1 2 0 3 1 2 2 2 4 3 1 1 1 1 5 3 3 2 1 1 1 0 0 1 2 3 4 3 1 3 3 3 3 3 4 1 0 3 1 3 5 3 4 5 3 3 3 1 3 1 3 5 4 4 3 1 1 3 2 5 1 3 3 3 0 2 5 4 4 1 1 3 0 1 1 5 2 5 5 0 1 1 2 2 3 1 1 4 5 3 4 1 3 5 1 3 2 3 1 1 4 1 2 4 2 2 1 1 3 2 1 4 2 2 3 0 4 1 0 1 1 2 2 0 5 2 2 1 2 2 4 1 2 1 3 3 0 5 3 4 3 3 4 0 4 1 3 1 1 2 0 1 1 4 4 0 3 3 1 2 5 2 5 1 2 4 4 1 3 5 2 3 5 1 4 0 2 4 1 1 1 4 3 4 1 3 1 2 4 4 5 1 2 5 3 3 1 4 1 3 3 4 5 3 3 3 3 4 4 4 1 0 5 1 5 1 5 3 3 3 5 1 1 3 2 1 4 1 4 3 3 5 2 4 1 1 1 0 4 1 4 1 1 3 1 2 3 1 1 1 2 0 3 1 3 1 0 1 5 5 4 3 4 5 2 1 3 1 1 4 1 3 3 3 1 5 5 2 1 1 1 3 3 1 2 4 3 2 0 1 4 5 2 3 3 4 2 2 1 1 2 1 3 5 3 5 2 4 4 1 1 4 4 3 4 3 1 1 3 3 2 4 3 3 2 1 3 4 2 1 3 2 3 2 3 3 1 2 1 3 4 3 4 3 3 1 5 2 3 4 4 3 1 3 2 3 2 4 1 4 3 2 2 1 3 3 2 3 3 1 2 1 2 0 1 1 1 5 5 3 2 1 3 4 3 4 

In [28]:
from sklearn import metrics
 
print(metrics.confusion_matrix(Y,Pred))

[[ 42  24   8  14   1   3]
 [  7 153  33  28  24   5]
 [  6  45  88  41  23  11]
 [  0  47  29 130  43  18]
 [  0  25  24  63 116  21]
 [  4  30  14  29  45  89]]


In [29]:
target_names = ['Pants', 'False', 'Barely-True','Half-True','Mostly-True','True']

print(metrics.classification_report(Y, Pred,target_names =target_names))

              precision    recall  f1-score   support

       Pants       0.71      0.46      0.56        92
       False       0.47      0.61      0.53       250
 Barely-True       0.45      0.41      0.43       214
   Hlaf-True       0.43      0.49      0.45       267
 Mostly-True       0.46      0.47      0.46       249
        True       0.61      0.42      0.50       211

    accuracy                           0.48      1283
   macro avg       0.52      0.48      0.49      1283
weighted avg       0.50      0.48      0.48      1283

